## Exercícios de Importação BD Sakila – Carga Incremental

1.Criar a tabela cp_rental_append, contendo a cópia da tabela rental com os campos rental_id e rental_date

In [ ]:
use sakila;

create table cp_rental_append select rental_id, rental_date from rental;

2.Criar a tabela cp_rental_id e cp_rental_date, contendo a cópia da tabela cp_rental_append

In [ ]:
create table cp_rental_id select * from cp_rental_append;

create table cp_rental_date select * from cp_rental_append;

Realizar com uso do Sqoop - Importações no warehouse /user/hive/warehouse/db_test3 e visualizar no HDFS

3.Importar as tabelas cp_rental_append, cp_rental_id e cp_rental_date com 1 mapeador

In [ ]:
sqoop import --connect jdbc:mysql://database/sakila --username root -password secret --wareho
use-dir /user/hive/warehouse/db_test3 -m 1 --table cp_rental_append

sqoop import --connect jdbc:mysql://database/sakila --username root -password secret --warehouse-dir /user/hive/warehouse/db_test3 -m 1 --table cp_rental_id

sqoop import --connect jdbc:mysql://database/sakila --username root -password secret --warehouse-dir /user/hive/warehouse/db_test3 -m 1 --table cp_rental_date

Realizar com uso do MySQL

4.Executar o sql /db-sql/sakila/insert_rental.sql no container do database

$ docker exec -it database bash

$ cd /db-sql/sakila

$ mysql -psecret < insert_rental.sql

Realizar com uso do Sqoop - Importações no warehouse /user/hive/warehouse/db_test3 e visualizar no HDFS

5.Atualizar a tabela cp_rental_append no HDFS anexando os novos arquivos

In [ ]:
sqoop import --table cp_rental_append --connect jdbc:mysql://database/sakila --username root -password secret --warehouse-dir /user/hive/warehouse/db_test3 -m 1 --append

6.Atualizar a tabela cp_rental_id no HDFS de acordo com o último registro de rental_id, adicionando apenas os novos dados.

In [ ]:
#descobrindo o último registro de rental_id e de rental_date para adicionar novos dados a partir deles
sqoop eval --connect jdbc:mysql://database/sakila --username root -password secret --query "select * from cp_rental_append order by rental_id desc limit 5"

-------------------------------------
| rental_id   | rental_date         |
-------------------------------------
| 16049       | 2005-08-23 22:50:12.0 |
| 16048       | 2005-08-23 22:43:07.0 |
| 16047       | 2005-08-23 22:42:48.0 |
| 16046       | 2005-08-23 22:26:47.0 |
| 16045       | 2005-08-23 22:25:26.0 |
-------------------------------------

sqoop import --table cp_rental_id --connect jdbc:mysql://database/sakila --username root -password secret --warehouse-dir /user/hive/warehouse/db_test3 -m 1 --incremental append --check-column rental_id --last-value 16049

7.Atualizar a tabela cp_rental_date no HDFS de acordo com o último registro de rental_date, atualizando os registros a partir desta data.

In [ ]:
sqoop import --table cp_rental_date --connect jdbc:mysql://database/sakila --username root -password secret --warehouse-dir /user/hive/warehouse/db_test3 -m 1 --incremental lastmodified --merge-key rental_id --check-column rental_date --last-value '2005-08-23 22:50:12.0'

## Importação para o Hive e Exportação - BD Employees

1.Importar a tabela employees.titles do MySQL para o diretório /user/aluno/<nome>/data com 1 mapeador.

In [ ]:
sqoop import --table titles --connect jdbc:mysql://database/employees --username root --password secret --warehouse-dir /user/aluno/esther/data -m 1

2.Importar a tabela employees.titles do MySQL para uma tabela Hive no banco de dados seu nome com 1 mapeador.

In [ ]:
sqoop import --table titles --connect jdbc:mysql://database/employees --username root --password secret -m 1 --hive-import --hive-table esther.titles

3.Selecionar os 10 primeiros registros da tabela titles no Hive.

In [ ]:
docker exec -it hive-server bash

beeline -u jdbc:hive2://localhost:10000

select * from titles limit 10;

+----------------+------------------+-------------------+-----------------+
| titles.emp_no  |   titles.title   | titles.from_date  | titles.to_date  |
+----------------+------------------+-------------------+-----------------+
| 10001          | Senior Engineer  | 1986-06-26        | 9999-01-01      |
| 10002          | Staff            | 1996-08-03        | 9999-01-01      |
| 10003          | Senior Engineer  | 1995-12-03        | 9999-01-01      |
| 10004          | Engineer         | 1986-12-01        | 1995-12-01      |
| 10004          | Senior Engineer  | 1995-12-01        | 9999-01-01      |
| 10005          | Senior Staff     | 1996-09-12        | 9999-01-01      |
| 10005          | Staff            | 1989-09-12        | 1996-09-12      |
| 10006          | Senior Engineer  | 1990-08-05        | 9999-01-01      |
| 10007          | Senior Staff     | 1996-02-11        | 9999-01-01      |
| 10007          | Staff            | 1989-02-10        | 1996-02-11      |
+----------------+------------------+-------------------+-----------------+

4.Deletar os registros da tabela employees.titles do MySQL e verificar se foram apagados, através do Sqoop

In [ ]:
 sqoop eval --connect jdbc:mysql://database/employees --username root --password secret --query "truncate table titles"

 sqoop eval --connect jdbc:mysql://database/employees --username root --password secret --query "select * from titles limit 10"

 ----------------------------------------------------------------
| emp_no      | title                | from_date  | to_date    |
----------------------------------------------------------------
----------------------------------------------------------------

5.Exportar os dados do diretório /user/hive/warehouse/<nome>.db/data/titles para a tabela do MySQL  employees.titles.

In [ ]:
sqoop export --table titles --connect jdbc:mysql://database/employees --username root --passw
ord secret --export-dir /user/aluno/esther/data/titles

6.Selecionar os 10 primeiros registros registros da tabela employees.titles do MySQL.

In [ ]:
sqoop eval --connect jdbc:mysql://database/employees --username root --password secret --query "select * from titles limit 10"

----------------------------------------------------------------
| emp_no      | title                | from_date  | to_date    |
----------------------------------------------------------------
| 10001       | Senior Engineer      | 1986-06-26 | 9999-01-01 |
| 10002       | Staff                | 1996-08-03 | 9999-01-01 |
| 10003       | Senior Engineer      | 1995-12-03 | 9999-01-01 |
| 10004       | Engineer             | 1986-12-01 | 1995-12-01 |
| 10004       | Senior Engineer      | 1995-12-01 | 9999-01-01 |
| 10005       | Senior Staff         | 1996-09-12 | 9999-01-01 |
| 10005       | Staff                | 1989-09-12 | 1996-09-12 |
| 10006       | Senior Engineer      | 1990-08-05 | 9999-01-01 |
| 10007       | Senior Staff         | 1996-02-11 | 9999-01-01 |
| 10007       | Staff                | 1989-02-10 | 1996-02-11 |
----------------------------------------------------------------